In [1]:
# %pylab

import numpy
import matplotlib
from matplotlib import pylab, mlab, pyplot
from mpl_toolkits.mplot3d import Axes3D
np = numpy
plt = pyplot

from pylab import *
from numpy import *

In [2]:
from scipy.optimize import curve_fit
import tkinter as tk
from tkinter import filedialog

matplotlib.rcParams['figure.figsize'] = [10.5,7]
matplotlib.rcParams.update({'font.size': 25})
%matplotlib

Using matplotlib backend: TkAgg


In [3]:
def gaussian(x, A, mu, sig):
    return A*exp(-power(x - mu, 2.) / (2 * power(sig, 2.)))

In [4]:
matplotlib.rcParams.update({'font.size': 25})

In [5]:
# sélection des fichiers (Python 3)
import tkinter as tk
from tkinter import filedialog

root = tk.Tk()
root.withdraw()

file_path = filedialog.askopenfilename(initialdir = '/home/adrian/Documents/Mesures',multiple=True)
print(file_path)

# ('/home/adrian/Documents/Mesures/GiantMol_Ions/20180606/fluo_temp_scan866__000_000.dat',
# '/home/adrian/Documents/Mesures/GiantMol_Ions/20180606/fluo_temp_scan866__000_001.dat',
# '/home/adrian/Documents/Mesures/GiantMol_Ions/20180606/fluo_temp_scan866__000_002.dat')

# ('/home/adrian/Documents/Mesures/laser_scan/20210416/fluo_dip_re_000.dat', '/home/adrian/Documents/Mesures/laser_scan/20210416/fluo_dip_re_001.dat')

('/home/adrien/Documents/Mesures/laser_scan/20220706/scan397_000.dat', '/home/adrien/Documents/Mesures/laser_scan/20220706/scan397_001.dat')


In [ ]:
('/home/adrian/Documents/Mesures/laser_scan/20210416/fluo_dip_re_000.dat', '/home/adrian/Documents/Mesures/laser_scan/20210416/fluo_dip_re_001.dat')

In [6]:
file_path

('/home/adrien/Documents/Mesures/laser_scan/20220706/scan397_000.dat',
 '/home/adrien/Documents/Mesures/laser_scan/20220706/scan397_001.dat')

In [7]:
# Array rectangle
# Allocation à l'avance

filename=file_path
m = loadtxt(file_path[0],usecols=(2),delimiter='\t',skiprows=0,unpack=True).size
n = len(file_path)
shape = (m, n)

TIME = zeros(shape)
PM = zeros(shape)
CH2 = zeros(shape)

for k in range(0,len(filename)):
    TIME[:,k], PM[:,k] = loadtxt(file_path[k],usecols=(0,1),delimiter='\t',skiprows=0,unpack=True)

In [13]:
# plot for simple scan

to_plot = 0
print(file_path[to_plot])
fig = figure('One res_noire',clear='True')
ax = fig.add_subplot(1, 1, 1)

ax.plot(TIME[:,to_plot],PM[:,to_plot],'+',color='xkcd:prussian blue',label='cloud fluo.')

xlabel(r'$\Delta \nu_{397}$ [MHz]')
ylabel('fluo. [cts/5ms]')
ax.legend(fontsize=18)

grid()
tight_layout()

/home/adrien/Documents/Mesures/laser_scan/20220706/scan397_000.dat


In [8]:
fond = 0
PM = PM - fond

In [9]:
# en longueurs d'onde
n = 1.0002746
c = 299792458
l397_1 = 397.70012*1e-9
l397_2 = 397.69972*1e-9
print(l397_1*1e9,l397_2*1e9)

l_397 = linspace(l397_2*1e9,l397_1*1e9,len(TIME[:,0])) # nm
f_397 = c/(n*l_397*1e-9) # Hz

397.70012 397.69972000000007


In [109]:
# en fréquence
n = 1.0002746
c = 299792458
f397_1 = 755.22170*1e12
f397_2 = 755.22245*1e12

f_397 = linspace(f397_1,f397_2,len(TIME[:,0])) # nm

In [41]:
test_gauss = gaussian(TIME[:,0],1500,15,3)
fit_PM = zeros((len(file_path),3))
fit_PM = zeros((len(file_path),3))
covar_PM = [zeros((3,3)) for k in range(0,len(file_path))]

A = zeros(len(file_path))
mu = zeros(len(file_path))
sig = zeros(len(file_path))
            
fit_gauss = zeros((len(TIME[:,0]),(len(file_path))))

start = 0

In [27]:
import matplotlib.patches as patches

In [110]:
# plot for simple scan

to_plot = 1
print(file_path[to_plot])
f_755 = 755.222765897*1e12
bckg =  min(PM[:,to_plot])
start = 0
stop = -1

start_exclude = 670
stop_exclude = 780
x = (f_397[start:]-f_755)*1e-9
fig = figure('One res_noire 1',clear='True')
ax = fig.add_subplot(1, 1, 1)


# plot(l_866,PM[:,to_plot],'+',color='xkcd:prussian blue')
# ax.plot(tata,PM[start:,to_plot],'+',color='xkcd:prussian blue',label='cloud fluo.')
ax.plot(x,PM[start:,to_plot],'+',color='xkcd:prussian blue',label='cloud fluo.') #tata
xlabel(r'$\Delta \nu_{397}$ [GHz]')
ylabel('fluo. [cts/5ms]')
# ax.legend(fontsize=18)

grid()
tight_layout()

/home/adrien/Documents/Mesures/laser_scan/20220706/scan397_001.dat


In [104]:
# plot for simple scan

to_plot = 0
print(file_path[to_plot])
f_755 = 755.222765897*1e12
bckg =  min(PM[:,to_plot])
start = 0
stop = -1

start_exclude = 670
stop_exclude = 780
x = (f_397[start:]-f_755)*1e-9
x_fit = linspace(x[0],x[-1],50000)

PMPM = hstack((PM[start:start_exclude,to_plot],PM[stop_exclude:,to_plot]))
xx =  hstack((x[:start_exclude],x[stop_exclude:]))


fit_PM[to_plot,:], covar_PM[to_plot] = curve_fit(gaussian,xx,PMPM-bckg,
                                                    p0=[5000,-0.45,0.5],maxfev = 50000)

A[to_plot] = fit_PM[to_plot,0]
mu[to_plot] = fit_PM[to_plot,1]
sig[to_plot] = fit_PM[to_plot,2]
FWHM = 2*sqrt(2*log(2)) * sig[to_plot] # GHz
temperature = 40 * (FWHM*1e9/f_755/(7.16*1e-7))**2

print(A[to_plot],mu[to_plot],sig[to_plot])
fit_gauss[:,to_plot] = gaussian(x,A[to_plot],mu[to_plot],sig[to_plot])
my_fit_gauss = gaussian(x,A[to_plot],mu[to_plot],sig[to_plot])

fig = figure('One res_noire',clear='True')
ax = fig.add_subplot(1, 1, 1)

# tata = (x[::-1])*1000 # MHz
tete = my_fit_gauss
# tete = tete + bckg
# x_fit = x_fit[::-1]


# plot(l_866,PM[:,to_plot],'+',color='xkcd:prussian blue')
# ax.plot(tata,PM[start:,to_plot],'+',color='xkcd:prussian blue',label='cloud fluo.')
ax.plot(x,PM[start:,to_plot],'+',color='xkcd:prussian blue',label='cloud fluo.') #tata
ax.plot(x,tete+bckg,color='xkcd:vermillion',label='Gaussian fit')
# vlines(mu[to_plot]*1000,0,15000,linestyles=':',color='xkcd:vermillion')    

rect = patches.Rectangle((x[start_exclude], 5500), (x[stop_exclude]-x[start_exclude]), 12000-5500,
                         linewidth=1, edgecolor='black', facecolor='gray',alpha=.3,hatch='xxxx',label='fit exclusion')
# Add the patch to the Axes
ax.add_patch(rect)

    
text( -1, 9500, f'FWHM = ${2*sqrt(2*log(2)) * sig[to_plot] * 1000:.0f}$ MHz\nT = {temperature:.0f} K',
rotation=0,
horizontalalignment='left',
verticalalignment='bottom',
color='xkcd:vermillion',
fontsize=17)

# ax.set_xlim(-1000,1050)
# ax.set_ylim(2800,15200)
xlabel(r'$\Delta \nu_{397}$ [GHz]')
ylabel('fluo. [cts/5ms]')
ax.legend(fontsize=18)

grid()
tight_layout()

/home/adrien/Documents/Mesures/laser_scan/20220706/scan397_000.dat
4463.510541837598 -0.35673391698750473 0.19079357047463197


In [72]:
temperature

538.2817631047924

In [45]:
# plot with complicated scan plot only crystal part at the end

to_plot = 1
print(file_path[to_plot])
f_755 = 755222.765897

start = 0
stop = -1

start_exclude = 0
stop_exclude = 1350
x = f_397-f_755

tata = (x[::-1]) # Ghz
x_fit = linspace(tata[stop_exclude-300],tata[-1],5000)

xx = concatenate(( tata[:start_exclude],tata[stop_exclude:] ))
PMPM = concatenate(( PM[:start_exclude,to_plot],PM[stop_exclude:,to_plot] ))

bckg = min(PMPM)
fit_PM[to_plot,:], covar_PM[to_plot] = curve_fit(gaussian,xx,PMPM,
                                                    p0=[5000,-1613.6,0.05],maxfev = 50000)

A[to_plot] = fit_PM[to_plot,0]
mu[to_plot] = fit_PM[to_plot,1]
sig[to_plot] = fit_PM[to_plot,2]
FWHM = 2*sqrt(2*log(2)) * sig[to_plot] * 1000 # MHz
temperature = 40 * (FWHM*1e-3/f_755/(7.16*1e-7))**2

print(A[to_plot],mu[to_plot],sig[to_plot])
# fit_gauss[:,to_plot] = gaussian(x,A[to_plot],mu[to_plot],sig[to_plot])
my_fit_gauss = gaussian(x_fit,A[to_plot],mu[to_plot],sig[to_plot])

f_397 = c/(1.0002746*l_397*1e-9) * 1e-9 # GHz

x_offset = 1614

fig = figure('One res_noire',clear='True')
ax = fig.add_subplot(1, 1, 1)

# plot(l_866,PM[:,to_plot],'+',color='xkcd:prussian blue')
ax.plot((tata+x_offset)*1e3,PM[start:,to_plot],'+',color='xkcd:prussian blue',label='cloud fluo.')
# ax.plot(xx,PMPM,'4',color='k',label='cloud fluo.')
ax.plot((x_fit+x_offset)*1e3,my_fit_gauss+bckg,color='xkcd:vermillion',label='Gaussian fit')
# vlines(mu[to_plot]*1000,0,15000,linestyles=':',color='xkcd:vermillion')    

rect = patches.Rectangle(((tata[start_exclude]+x_offset)*1e3, 0), (tata[stop_exclude]-tata[start_exclude])*1e3, 16000,
                         linewidth=1, edgecolor='black', facecolor='gray',alpha=.3,hatch='xxxx',label='fit exclusion')
# Add the patch to the Axes
ax.add_patch(rect)


ax.annotate("", xy=((-1613.689+x_offset)*1e3,2000), xytext=((-1613.689+x_offset)*1e3,3500),
            arrowprops=dict(width=1,facecolor='k'))
# rect = patches.Rectangle((-1613.7, 0), (0.025), 16000,
#                          linewidth=1, edgecolor='black', facecolor='gray',alpha=.3,hatch='',label='fit exclusion')
# # Add the patch to the Axes
# ax.add_patch(rect)

    
text((-1613.87+x_offset)*1e3, 3000, f'FWHM = ${2*sqrt(2*log(2)) * sig[to_plot] * 1000:.0f}$ MHz\n        T = {temperature:.0f} K',
rotation=0,
horizontalalignment='left',
verticalalignment='bottom',
color='xkcd:vermillion',
fontsize=17,
     bbox=dict(facecolor='white'))

ax.set_xlim((-1614.37+x_offset)*1e3,(-1613.55+x_offset)*1e3)
# ax.set_ylim(2800,15200)
xlabel(r'$\Delta \nu_{397} +$' + f'{x_offset*1e3:.0f}' + ' [MHz]')
ylabel('fluo. [cts/5ms]')
ax.legend(fontsize=18)

grid()
tight_layout()

savefig('397scan.png',dpi=300)

/home/adrian/Documents/Mesures/laser_scan/20210416/fluo_dip_re_001.dat
4359.215689935037 -1613.5975316223546 0.032963022813756876


In [19]:
figure('coucou',clear='True')

plot(xx,PMPM,'+',ls='--')

In [398]:
f_346 = 346000.234867
# FWHM = 1.75*1e3
temperature = 40 * (FWHM*1e-3/f_346/(7.16*1e-7))**2
print(temperature)

3.9269074839873697


In [59]:
tete

array([2590., 2590., 2590., ..., 2590., 2590., 2590.])